# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import logging
import os
import csv
import joblib

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core import Workspace, Experiment, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from pprint import pprint # Used in printing automl model parameters
from azureml.core import Model # Used to get model information

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl-heartfailure'
experiment = Experiment(ws, experiment_name)

print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

run = experiment.start_logging()

In [ ]:
# choose a name for your cluster
# Compute name should contain only letters, digits, hyphen and should be 2-16 charachters long
cluster_name = "hd-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print(f'{cluster_name} exists already')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    
    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    
    compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
    
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
found = False
key = "Heart-Failure Dataset"
description_text = "Heart-Failure Dataset for Udacity Course 2"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/PeacePeters/Deploy-the-best-model-using-AzureML/main/heart_failure.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

df = dataset.to_pandas_dataframe()
df.describe()

In [ ]:
# Review the dataset result
dataset.take(5).to_pandas_dataframe()

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",   
                             path = project_folder,
                             featurization= "auto",
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [ ]:
# Submit your experiment
remote_run = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# Get best run and model
best_run, fitted_model = automl_run.get_output()
print(best_run)
print(fitted_model)

In [ ]:
# Get all metrics of the best run
best_run_metrics = best_run.get_metrics()

# Print all metrics of the best run
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

In [ ]:
# Print detailed parameters of the fitted model
def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(fitted_model)

In [ ]:
best_run.get_tags()

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# Register the model
model = best_run.register_model(model_path='outputs/fitted_model.pkl', model_name='heart_failure_automl_model',
                   tags={'Training context':'AutoML'}
                   properties={'AUC_weighted': best_run_metrics['AUC_weighted']})
model

In [ ]:
model = best_run.register_model(model_path='outputs/fitted_model.joblib', model_name='heart_failure_automl_model',
                   tags={'Training context':'AutoML', 'type': 'Classification'},
                   properties={'AUC_weighted': best_run_metrics['AUC_weighted']})
model

In [ ]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script="score.py", environment=best_run.get_environment())

aci_config = AciWebservice.deploy_configuration(cpu_cores=1,
                                               memory_gb=1,
                                               enable_app_insights=True, 
                                               tags={'name': 'aci-cluster', 'framework': 'AutoML'},
                                               description='Heart Failure Predictor Web Service')

service = Model.deploy(workspace=ws, 
                           name='heart-failure-srv', 
                           models=[model], 
                           inference_config=inference_config, 
                           deployment_config=aci_config)

service.wait_for_deployment(True)
print(service.state)

In [ ]:
from azureml.core.webservice import LocalWebservice

local_config=LocalWebservice.deploy_configuration(port=9000)

local_service = Model.deploy(workspace=ws, 
                           name='test', 
                           models=[model], 
                           inference_config=inference_config, 
                           deployment_config=local_config)

local_service.wait_for_deployment(True)
print(local_service.state)

In [ ]:
service = Model.deploy(workspace=ws,
                       name='heart-failure-srv',
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

In [ ]:
import json

# import test data
test_df = df.sample(5) # data is the pandas dataframe of the original data
label_df = test_df.pop('DEATH_EVENT')

test_sample = json.dumps({'dataset': test_df.to_dict(orient='records')})
print(test_sample)

# predict using the deployed model
result = service.run(json.dumps(test_sample))
print(result)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
# Set the content type
headers = {'Content-type': 'application/json'}

response = requests.post(service.scoring_uri, test_sample, headers=headers)

# Print results from the inference
print(response.text)

In [ ]:
# Print original labels
print(label_df)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())

In [ ]:
service.delete()